In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
# Define input shape and number of classes
input_shape = (256, 256, 3)  # New input size
num_classes = 1  # Binary classification (attentive vs. distracted)

# Function to build a model
def build_model(base_model, input_shape):
    base_model = base_model(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False #Freezing base layers
        
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Load pre-trained models
vgg16_model = build_model(VGG16, input_shape)
resnet50_model = build_model(ResNet50, input_shape)
inceptionv3_model = build_model(InceptionV3, input_shape)

# Compile models
vgg16_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
inceptionv3_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=40,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest',  # Fill missing pixels after transformations
    validation_split=0.2  # Split 20% of the data for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    'Collected_Dataset',  # Path to dataset
    target_size=input_shape[:2],  # Resize images to (256, 256)
    batch_size=16,
    class_mode='binary',
    subset='training',  # Use the training subset
    shuffle=True
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    'Collected_Dataset',  # Path to your dataset
    target_size=input_shape[:2],  # Resize images to (256, 256)
    batch_size=16,
    class_mode='binary',
    subset='validation',  # Use the validation subset
    shuffle=True
)

Found 202 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [21]:
# Common early stopping
earlystop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fine-tune models

# === VGG16 ===
checkpoint_vgg = ModelCheckpoint('best_model_vgg16.keras', monitor='val_accuracy', save_best_only=True)
print("Fine-tuning VGG16...")
vgg16_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[earlystop, checkpoint_vgg]
)

# === ResNet50 ===
checkpoint_resnet = ModelCheckpoint('best_model_resnet50.keras', monitor='val_accuracy', save_best_only=True)
print("Fine-tuning ResNet50...")
resnet50_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[earlystop, checkpoint_resnet]
)

# === InceptionV3 ===
checkpoint_inception = ModelCheckpoint('best_model_inceptionv3.keras', monitor='val_accuracy', save_best_only=True)
print("Fine-tuning InceptionV3...")
inceptionv3_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[earlystop, checkpoint_inception]
)


# Generate predictions on the validation set
print("Generating predictions...")
vgg16_preds = vgg16_model.predict(validation_generator)
resnet50_preds = resnet50_model.predict(validation_generator)
inceptionv3_preds = inceptionv3_model.predict(validation_generator)

# Convert predictions to binary (0 or 1)
vgg16_preds = [1 if pred > 0.5 else 0 for pred in vgg16_preds]
resnet50_preds = [1 if pred > 0.5 else 0 for pred in resnet50_preds]
inceptionv3_preds = [1 if pred > 0.5 else 0 for pred in inceptionv3_preds]

# Combine predictions using majority voting
ensemble_preds = []
for vgg_pred, resnet_pred, inception_pred in zip(vgg16_preds, resnet50_preds, inceptionv3_preds):
    votes = [vgg_pred, resnet_pred, inception_pred]
    majority_vote = max(set(votes), key=votes.count)
    ensemble_preds.append(majority_vote)

# Evaluate the ensemble
print("Evaluating ensemble...")
report = classification_report(validation_generator.classes, ensemble_preds, target_names=['Focused', 'Distracted'])
print(report)

Fine-tuning VGG16...
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.5604 - loss: 0.6980 - val_accuracy: 0.5200 - val_loss: 0.7118
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.5742 - loss: 0.7435 - val_accuracy: 0.5000 - val_loss: 0.7016
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.5958 - loss: 0.6653 - val_accuracy: 0.4800 - val_loss: 0.7139
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.6208 - loss: 0.6294 - val_accuracy: 0.4400 - val_loss: 0.7278
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.6439 - loss: 0.6195 - val_accuracy: 0.4400 - val_loss: 0.7342
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.5861 - loss: 0.7229 - val_accuracy: 0.5400 - val_loss: 0.6900
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.6403 - loss: 0.6523 - val_accuracy: 0.5600 - val_loss: 0.6975
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.6755 - loss: 0.6299 - val_accurac

In [ ]:
# Save VGG16 model
vgg16_model.save('vgg16_model.keras')

# Save ResNet50 model
resnet50_model.save('resnet50_model.keras')

# Save InceptionV3 model
inceptionv3_model.save('inceptionv3_model.keras')